In [1]:
import pandas as pd


from tqdm import tqdm

In [2]:
anima_df = pd.read_pickle('anima_translated_vicuna_eval_dataset_responses.pickle')

In [3]:
belle_df = pd.read_pickle('belle7b2m_translated_vicuna_eval_dataset_responses.pickle')

In [7]:
def extract_belle_answer(a):
    return a[a.index("\n\nAssistant:\n")+len("\n\nAssistant:\n"):]

In [8]:
belle_df['extracted_belle_answer'] = belle_df['belle_answer'].apply(extract_belle_answer)

In [10]:
chatgpt_df = pd.read_pickle('chatgpt_translated_vicuna_eval_dataset_responses.pickle')

In [12]:
cn_vicuna_df = pd.read_pickle('cn_vicuna_translated_vicuna_eval_dataset_responses.pickle')

In [21]:
def extract_cn_vicuna_answer(a):
    start_pos = a.index("### Response:\n")+len("### Response:\n")
    try:
        end_pos = a.index('\n\n##', start_pos)
    except Exception as e:
        end_pos = len(a)
    
    return a[start_pos:end_pos]

# gpt4 setup

In [25]:
import backoff
import openai
openai.api_key = 'KEY'

In [26]:

def run_gpt4(sys_prompt,user_prompt , n=1, oa_model_type='gpt-4', max_tokens=None):
    if max_tokens is None:
        completion = openai.ChatCompletion.create(model=oa_model_type,
                                                  n=n,
                                                  temperature=0.2, # follow vicuna setting
                                                  messages=[
                    {"role": "system", "content": sys_prompt},
                    {
                        "role": "user",
                        "content": user_prompt,
                    },                                                  ])
    else:
        completion = openai.ChatCompletion.create(model=oa_model_type,
                                                  n=n,
                                                  temperature=0.2, # follow vicuna setting
                                                  max_tokens=max_tokens,
                                                  messages=[
                    {"role": "system", "content": sys_prompt},
                    {
                        "role": "user",
                        "content": user_prompt,
                    },                                                  ])

    to_ret = []

    for c in completion['choices']:
        to_ret.append(c['message']['content'])
    return to_ret

In [27]:

@backoff.on_exception(backoff.expo, openai.error.RateLimitError,
                     max_tries=10,
                      raise_on_giveup=False,)
def run_gpt4_backoff(*args,**kwargs):
    return run_gpt4(*args,**kwargs)


In [28]:
run_gpt4_backoff('', '中国的首都是哪里？')[0]

'中国的首都是北京。'

# score and elo 

In [10]:
# borrow code from: https://github.com/lm-sys/FastChat/blob/main/fastchat/eval/eval_gpt_review.py

In [34]:
def parse_score(review):
    try:
        score_pair = review.split("\n")[0]
        score_pair = score_pair.replace(",", " ")
        sp = score_pair.split(" ")
        if len(sp) == 2:
            return [float(sp[0]), float(sp[1])]
        else:
            raise Exception("Invalid score pair.")
    except Exception as e:
        print(
            f"{e}\nContent: {review}\n" "You must manually fix the score pair."
        )
        return [-1, -1]

In [35]:
prompt_jsons = [
    {"prompt_id": 1, "system_prompt": "You are a helpful and precise assistant for checking the quality of the answer.", "prompt_template": "[Question]\n{question}\n\n[The Start of Assistant 1's Answer]\n{answer_1}\n\n[The End of Assistant 1's Answer]\n\n[The Start of Assistant 2's Answer]\n{answer_2}\n\n[The End of Assistant 2's Answer]\n\n[System]\n{prompt}\n\n", "defaults": {"prompt": "We would like to request your feedback on the performance of two AI assistants in response to the user question displayed above.\nPlease rate the helpfulness, relevance, accuracy, level of details of their responses. Each assistant receives an overall score on a scale of 1 to 10, where a higher score indicates better overall performance.\nPlease first output a single line containing only two values indicating the scores for Assistant 1 and 2, respectively. The two scores are separated by a space. In the subsequent line, please provide a comprehensive explanation of your evaluation, avoiding any potential bias and ensuring that the order in which the responses were presented does not affect your judgment."}, "description": "Prompt for general questions", "category": "general"},
    {"prompt_id": 2, "system_prompt": "You are a helpful and precise assistant for checking the quality of the answer.", "prompt_template": "[Question]\n{question}\n\n[The Start of Assistant 1's Answer]\n{answer_1}\n\n[The End of Assistant 1's Answer]\n\n[The Start of Assistant 2's Answer]\n{answer_2}\n\n[The End of Assistant 2's Answer]\n\n[System]\n{prompt}\n\n", "defaults": {"prompt": "Your task is to evaluate the coding abilities of the above two assistants. They have been asked to implement a program to solve a given problem. Please review their code submissions, paying close attention to their problem-solving approach, code structure, readability, and the inclusion of helpful comments.\n\nPlease ensure that the assistants' submissions:\n\n1. Correctly implement the given problem statement.\n2. Contain accurate and efficient code.\n3. Include clear and concise comments that explain the code's logic and functionality.\n4. Adhere to proper coding standards and best practices.\n\nOnce you have carefully reviewed both submissions, provide detailed feedback on their strengths and weaknesses, along with any suggestions for improvement. You should first output a single line containing two scores on the scale of 1-10 (1: no code/no sense; 10: perfect) for Assistant 1 and 2, respectively. Then give extra comments starting from the next line."}, "description": "Prompt for coding questions", "category": "coding"},
    {"prompt_id": 3, "system_prompt": "You are a helpful and precise assistant for checking the quality of the answer.", "prompt_template": "[Question]\n{question}\n\n[The Start of Assistant 1's Answer]\n{answer_1}\n\n[The End of Assistant 1's Answer]\n\n[The Start of Assistant 2's Answer]\n{answer_2}\n\n[The End of Assistant 2's Answer]\n\n[System]\n{prompt}\n\n", "defaults": {"prompt": "We would like to request your feedback on the mathematical proficiency of two AI assistants regarding the given user question displayed above.\nFirst, please solve the problem independently, without referring to the answers provided by Assistant 1 and Assistant 2.\nAfterward, please examine the problem-solving process of Assistant 1 and Assistant 2 step-by-step to ensure their correctness, identifying any incorrect steps if present. Your evaluation should take into account not only the answer but also the problem-solving steps.\nFinally, please output a Python tuple containing two numerical scores for Assistant 1 and Assistant 2, ranging from 1 to 10, respectively. If applicable, explain the reasons for any variations in their scores and determine which assistant performed better."}, "description": "Prompt for math questions", "category": "math"}
    ]

In [36]:
reviewer_jsons = [{"reviewer_id": "gpt-4-0328-default", "prompt_id": 1, "metadata": {"temperature": 0.2, "max_tokens": 1024}, "description": "GPT-4 for general questions", "category": "general"},
    {"reviewer_id": "gpt-4-0328-coding", "prompt_id": 2, "metadata": {"temperature": 0.2, "max_tokens": 1024}, "description": "GPT-4 for coding questions", "category": "coding"},
    {"reviewer_id": "gpt-4-0328-math", "prompt_id": 3, "metadata": {"temperature": 0.2, "max_tokens": 1024}, "description": "GPT-4 for math questions", "category": "math"}
    ]

In [37]:

def gen_prompt(reviewer_jsons, prompt_jsons, cat, ques, ans1, ans2):
    # Default to general category (index=0)
    reviewer_idx = 0
    for idx, reviewer in enumerate(reviewer_jsons):
        if reviewer["category"] == cat:
            reviewer_idx = idx
            break
    prompt_id = reviewer_jsons[reviewer_idx]["prompt_id"]
    prompt_json = prompt_jsons[prompt_id - 1]
    assert prompt_json["prompt_id"] == prompt_id

    sys_prompt = prompt_json["system_prompt"]
    prompt_template = prompt_json["prompt_template"]
    defaults = prompt_json["defaults"]
    prompt = prompt_template.format(
        question=ques, answer_1=ans1, answer_2=ans2, **defaults
    )

    return sys_prompt, prompt, reviewer_idx + 1


### tournanment

In [30]:
assert (anima_df['question_id'] == belle_df['question_id']).all()
assert (anima_df['question_id'] == cn_vicuna_df['question_id']).all()
assert (anima_df['question_id'] == chatgpt_df['question_id']).all()

In [40]:
import random
from collections import defaultdict



In [41]:
model_dfs = [
    {'model':'anima', 'df':anima_df, 'answer_col':'anima_answer_extraced'},
    {'model':'belle', 'df':belle_df, 'answer_col':'extracted_belle_answer'},
    {'model':'cn_vicuna', 'df':cn_vicuna_df, 'answer_col':'extracted_cn_vicuna_answer'},
    {'model':'chatgpt', 'df':chatgpt_df, 'answer_col':'chatgpt_answer'},
            ]

In [45]:
rounds = 300
K=32
seed=42
INIT_RATING=1000
SCALE=400
BASE=10
random.seed(seed)

records = []
rating = defaultdict(lambda: INIT_RATING)

In [ ]:

pbar = tqdm(total=rounds)
result_list = []

In [54]:

while len(records) < rounds:
    while True:
        random_row = random.randint(0,len(anima_df)-1)
        first_model = random.randint(0,3)
        second_model = random.randint(0,2)
        if second_model == first_model:
            second_model = (second_model + 1)%4
            
        if not (random_row, first_model, second_model) in records:
            break
        
    print(f"competing: {(random_row, model_dfs[first_model]['model'], model_dfs[second_model]['model'])}")
    
    
    
    sys_prompt, prompt, reviewer_idx = gen_prompt(reviewer_jsons, prompt_jsons, 
                                                  model_dfs[first_model]['df'].iloc[random_row]['category'],
                                                  model_dfs[first_model]['df'].iloc[random_row]['translation'], 
                                                  model_dfs[first_model]['df'].iloc[random_row][model_dfs[first_model]['answer_col']], 
                                                  model_dfs[second_model]['df'].iloc[random_row][model_dfs[second_model]['answer_col']])
            
    #print(sys_prompt, prompt)
    print(f"PROMPT: \nsys_prompt:\n{sys_prompt}\nprompt:\n{prompt}")
    res = run_gpt4_backoff(sys_prompt, prompt)[0]
    score = parse_score(res)
    
    if score[0] < 0 or score[1] < 0:
        print(f"bad score: {score}, skipping")
        continue
    
    to_append = {**model_dfs[first_model]['df'].iloc[random_row], 
                 **model_dfs[second_model]['df'].iloc[random_row], 
                 **{'score':score,
                    'row_id':random_row,
                    'first_model':model_dfs[first_model]['model'],
                    'second_model':model_dfs[second_model]['model'],
                   }}
    
    print(f"gpt4 output: {to_append}")
    result_list.append(to_append)
    records.append((random_row, first_model, second_model)) # add in the end in case bad score skipping
    
    # elo following vicuna code
    ra = rating[model_dfs[first_model]['model']]
    rb = rating[model_dfs[second_model]['model']]
    ea = 1 / (1 + BASE ** ((rb - ra) / SCALE))
    eb = 1 / (1 + BASE ** ((ra - rb) / SCALE))
    
    if score[0] > score[1]:
        sa = 1
    elif score[1] > score[0]:
        sa = 0
    elif score[0] == score[1]:
        sa = 0.5
    else:
        raise Exception(f"unexpected vote {win}")
    rating[model_dfs[first_model]['model']] += K * (sa - ea)
    rating[model_dfs[second_model]['model']] += K * (1 - sa - eb)
    
    print(f"elo rating updated: {rating}")
    print(f"records: {len(records)}")
    pbar.update()
    print("-"*100)
    

competing: (33, 'chatgpt', 'anima')
PROMPT: 
sys_prompt:
You are a helpful and precise assistant for checking the quality of the answer.
prompt:
[Question]
您如何判断一个人是真的对谈话感兴趣还是只是在礼貌地应对？

[The Start of Assistant 1's Answer]
作为AI语言模型，我无法判断一个人是否真的对谈话感兴趣，因为我没有情感和直觉。但是，有一些迹象可以表明一个人是否对谈话感兴趣，例如：

1. 他们会主动参与谈话，提出问题或分享自己的经验。

2. 他们会保持眼神接触，面带微笑，身体语言放松。

3. 他们会回应你的话题，提出相关的问题或评论。

4. 他们会倾听你说话，不会打断你或分心。

5. 他们会回应你的情感，表达同情或理解。

总之，如果一个人表现出对谈话的积极参与和关注，那么他们很可能是真的对谈话感兴趣。

[The End of Assistant 1's Answer]

[The Start of Assistant 2's Answer]
 通过以下方法来判断一个人是否真正对谈话感兴趣：
1. 看他/她的表情、身体语言和手势是否与内容相符合；
2. 注意到他/她是否有足够的知识和经验，能够进行深入的交流；
3. 观察他/她是否会提出自己的想法或者反驳别人的论点，不仅仅是盲目接受或者同意；
4. 考虑其他的非verbal通信，例如是否会眼神移动，是否会瞬间的微笑等等。

若果这些指标都达到了，则可以认为该人真正对谈话感兴趣。

[The End of Assistant 2's Answer]

[System]
We would like to request your feedback on the performance of two AI assistants in response to the user question displayed above.
Please rate the helpfulness, relevance, accuracy, level of details of their responses. Each assistan

 29%|██▊       | 86/300 [11:20<1:41:53, 28.57s/it]

gpt4 output: {'question_id': 34, 'text': 'How can you determine if a person is genuinely interested in a conversation or simply being polite?', 'category': 'common-sense', 'translation': '您如何判断一个人是真的对谈话感兴趣还是只是在礼貌地应对？', 'chatgpt_answer': '作为AI语言模型，我无法判断一个人是否真的对谈话感兴趣，因为我没有情感和直觉。但是，有一些迹象可以表明一个人是否对谈话感兴趣，例如：\n\n1. 他们会主动参与谈话，提出问题或分享自己的经验。\n\n2. 他们会保持眼神接触，面带微笑，身体语言放松。\n\n3. 他们会回应你的话题，提出相关的问题或评论。\n\n4. 他们会倾听你说话，不会打断你或分心。\n\n5. 他们会回应你的情感，表达同情或理解。\n\n总之，如果一个人表现出对谈话的积极参与和关注，那么他们很可能是真的对谈话感兴趣。', 'anima_answer': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n您如何判断一个人是真的对谈话感兴趣还是只是在礼貌地应对？请回答尽量详细和完善，详细的阐释回答的原因，并给出尽量完整的相关的背景信息。请使用中文回答问题。\n\n### Response:  通过以下方法来判断一个人是否真正对谈话感兴趣：\n1. 看他/她的表情、身体语言和手势是否与内容相符合；\n2. 注意到他/她是否有足够的知识和经验，能够进行深入的交流；\n3. 观察他/她是否会提出自己的想法或者反驳别人的论点，不仅仅是盲目接受或者同意；\n4. 考虑其他的非verbal通信，例如是否会眼神移动，是否会瞬间的微笑等等。\n\n若果这些指标都达到了，则可以认为该人真正对谈话感兴趣。', 'anima_answer_extraced': ' 通过以下方法来判断一个人是否真正对谈话感兴趣：\n1. 看他/她的表情、身体语言和手势

 29%|██▉       | 87/300 [11:44<1:40:26, 28.29s/it]

gpt4 output: {'question_id': 39, 'text': "Do we have a moral obligation to explore space, or should we focus on solving Earth's problems first?", 'category': 'common-sense', 'translation': '我们是否有道德义务去探索太空，还是应该先集中精力解决地球上的问题？', 'chatgpt_answer': '这是一个复杂的问题，没有简单的答案。以下是一些可能需要考虑的因素：\n\n1. 道德义务：我们是否有道德义务去探索太空？这取决于我们对道德义务的理解。有些人认为，我们有责任探索太空，因为这是我们作为人类的使命之一。其他人则认为，我们应该优先解决地球上的问题，因为这是我们最紧迫的责任。\n\n2. 资源分配：探索太空需要大量的资源，包括资金、人力和技术。如果我们把这些资源用于解决地球上的问题，可能会更有效地改善人类的生活。然而，如果我们不探索太空，我们可能会错失一些重要的发现和机会。\n\n3. 科学发现：探索太空可以带来许多科学发现，这些发现可能会对地球上的问题有所帮助。例如，我们可以通过研究其他星球和行星来了解地球的起源和演化。此外，太空技术也可以用于解决地球上的问题，例如气候变化和能源问题。\n\n4. 风险和安全：探索太空也存在一定的风险和安全问题。太空探索可能会导致卫星碰撞、太空垃圾等问题，这些问题可能会对地球上的人类造成威胁。此外，太空探索也可能会导致军事竞赛和武器化的风险。\n\n综上所述，我们应该综合考虑以上因素，平衡探索太空和解决地球上问题的需求。我们可以通过投资科学研究和技术创新来推动太空探索，同时也应该优先解决地球上的紧迫问题，例如贫困、疾病和环境问题。', 'belle_answer': 'Human: \n我们是否有道德义务去探索太空，还是应该先集中精力解决地球上的问题？\n\nAssistant:\n\n这个问题没有一个明确的答案。一些人认为探索太空是重要的科学和技术发展，可以带来许多好处，例如提高我们对宇宙的了解、开发新技术和创造就业机会。但是，也有人认为我们应该将更多的时间和资源用于解决当前的社会和环境问题，例如贫困、饥饿、疾病和气候变化等。这是一个需要平

...

gpt4 output: {'question_id': 65, 'text': 'Implement a binary search algorithm to find a specific element in a sorted array.', 'category': 'coding', 'translation': '问题：实现一个二分搜索算法，在一个已排序的数组中查找特定元素。', 'belle_answer': 'Human: \n问题：实现一个二分搜索算法，在一个已排序的数组中查找特定元素。\n\nAssistant:\n\n解决方案：\n1. 遍历数组，将第一个位置和最后一个位置标记为已访问。\n2. 对于每个位置，计算中间位置的索引，并将其作为目标值与中间值进行比较。\n3. 如果目标值等于中间值，则返回该位置。\n4. 如果目标值小于中间值，则继续在左侧递归查找；如果大于中间值，则继续在右侧递归查找。\n5. 重复步骤2-4直到找到目标值或者确认没有目标值。', 'extracted_belle_answer': '\n解决方案：\n1. 遍历数组，将第一个位置和最后一个位置标记为已访问。\n2. 对于每个位置，计算中间位置的索引，并将其作为目标值与中间值进行比较。\n3. 如果目标值等于中间值，则返回该位置。\n4. 如果目标值小于中间值，则继续在左侧递归查找；如果大于中间值，则继续在右侧递归查找。\n5. 重复步骤2-4直到找到目标值或者确认没有目标值。', 'cn_vicuna_answer': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n问题：实现一个二分搜索算法，在一个已排序的数组中查找特定元素。\n\n### Response:\n\n```\n  public int binarySearch(int[] arr, int target) {\n    if (arr == null || arr.length < 1) return -1; // 如果数组为空或者长度小于等于0则返回-1\n    for (int i =

100%|█████████▉| 299/300 [1:39:06<00:23, 23.41s/it]

gpt4 output: {'question_id': 4, 'text': 'How can I increase my productivity while working from home?', 'category': 'generic', 'translation': '在家工作时，我如何提高我的工作效率？', 'anima_answer': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n在家工作时，我如何提高我的工作效率？请回答尽量详细和完善，详细的阐释回答的原因，并给出尽量完整的相关的背景信息。请使用中文回答问题。\n\n### Response:  以下是提高在家工作效率的方法：\n1. 制定清晰的计划表：先根据自己的任务列表、休息时间等情况，制定一个明确的计划表，每天都要有目标，不能拖延任务。\n2. 分配好时间：按照计划表分配好时间来完成任务，避免被其他事情干扰到你的工作上。\n3. 保持正常的生活习惯：保证充足的睡眠、饮食、运动等基本需求，这样才能保证身体健康，更加稳重地进行工作。\n4. 避免外部干扰：尽可能避免接受电话打扰或者与人交流，只有当前没有任何事情需要处理之后才会开始接听电话或者与人交流。\n5. 利用技术支持：通过使用各种工具（例如待办事项管理器、时间管理器等），帮助自己更好地管理工作，提高效率。\n6. 培养自律性：尤其是在家里工作的人，必须培养自律性，不要因为玩游戏、看视频等消极影响工作。', 'anima_answer_extraced': ' 以下是提高在家工作效率的方法：\n1. 制定清晰的计划表：先根据自己的任务列表、休息时间等情况，制定一个明确的计划表，每天都要有目标，不能拖延任务。\n2. 分配好时间：按照计划表分配好时间来完成任务，避免被其他事情干扰到你的工作上。\n3. 保持正常的生活习惯：保证充足的睡眠、饮食、运动等基本需求，这样才能保证身体健康，更加稳重地进行工作。\n4. 避免外部干扰：尽可能避免接受电话打扰或者与人交流，只有当前没有任何事情需要处理之后才会开始接听电话或者与人交流。\n5. 利用

100%|██████████| 300/300 [1:39:20<00:00, 20.63s/it]

gpt4 output: {'question_id': 52, 'text': 'What if the Aztecs had successfully repelled the Spanish conquistadors?', 'category': 'counterfactual', 'translation': '如果阿兹特克人成功抵挡住了西班牙征服者，会怎么样？', 'chatgpt_answer': '如果阿兹特克人成功抵挡住了西班牙征服者，可能会有以下几种情况：\n\n1. 阿兹特克帝国继续独立存在：如果阿兹特克人成功抵挡住了西班牙征服者，他们可能会继续独立存在，维持自己的文化和政治制度。\n\n2. 西班牙人寻找其他目标：如果西班牙征服者无法征服阿兹特克帝国，他们可能会寻找其他目标，例如印加帝国或其他南美洲的文明。\n\n3. 阿兹特克人与西班牙人和平共存：如果阿兹特克人和西班牙人能够和平共存，他们可能会建立贸易和文化交流关系，从而促进两个文明的发展。\n\n4. 阿兹特克人与其他欧洲国家建立关系：如果阿兹特克人成功抵挡住了西班牙征服者，他们可能会与其他欧洲国家建立关系，例如英国、法国或荷兰，从而获得更多的贸易和文化交流机会。', 'cn_vicuna_answer': "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n如果阿兹特克人成功抵挡住了西班牙征服者，会怎么样？\n\n### Response:\nIf Aitichel people successfully resisted against Spanish invaders, how would it be?\n\n## See also\n\n* Chinese language\n  * Classical Chinese\n    * Han characters (漢字/汉字; 漢⁄韩字; Hànzì)\n      * Simplified Chinese characters (简体字/簡體字; Jiǎntǐzì)\n        * Traditional Chinese